In [1]:
from PIL import ImageFilter, ImageStat, Image, ImageDraw
from multiprocessing import Pool, cpu_count
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import numpy as np
import glob
import cv2


In [2]:
cpu_count()

6

In [4]:
def im_multi(path):
    try:
        im_stats_im_ = Image.open(path)
        return [path, {'size': im_stats_im_.size}]
    except:
        print(path)
        return [path, {'size': [0,0]}]

def im_stats(im_stats_df):
    im_stats_d = {}
    p = Pool(cpu_count())
    ret = p.map(im_multi, im_stats_df['path'])
#     print(len(ret))
    for i in range(len(ret)):
#         if i % 20 == 0:
#             print(i)
        im_stats_d[ret[i][0]] = ret[i][1]
    im_stats_df['size'] = im_stats_df['path'].map(lambda x: ' '.join(str(s) for s in im_stats_d[x]['size']))
    return im_stats_df

def get_im_cv2(path):
    img = cv2.imread(path)
    resized = cv2.resize(img, (224, 224), cv2.INTER_LINEAR) #use cv2.resize(img, (64, 64), cv2.INTER_LINEAR)
    print('Processed yet another image')
    return [path, resized]

def normalize_image_features(paths):
    print('here')
    imf_d = {}
    p = Pool(cpu_count())
    print('here_1')
    ret = p.map(get_im_cv2, paths)
    print(len(ret))
    for i in range(len(ret)):
        if i % 20 == 0:
            print(i)        
        imf_d[ret[i][0]] = ret[i][1]
    ret = []
    fdata = [imf_d[f] for f in paths]
    fdata = np.array(fdata, dtype=np.uint8)
#     fdata = fdata.transpose((0, 3, 1, 2))
    fdata = fdata.astype('float32')
    fdata = fdata / 255
    return fdata

def subtract_mean(data, mean):
    for sample in data:
        sample[0] -= mean[0]
        sample[1] -= mean[1]
        sample[2] -= mean[2]
    return data


In [2]:

def im_multi(path):
    try:
        im_stats_im_ = Image.open(path)
        return [path, {'size': im_stats_im_.size}]
    except:
        print(path)
        return [path, {'size': [0,0]}]

def im_stats(im_stats_df):
    im_stats_d = {}
    p = Pool(cpu_count())
    ret = p.map(im_multi, im_stats_df['path'])
    for i in range(len(ret)):
        im_stats_d[ret[i][0]] = ret[i][1]
    im_stats_df['size'] = im_stats_df['path'].map(lambda x: ' '.join(str(s) for s in im_stats_d[x]['size']))
    return im_stats_df

def get_im_cv2(path):
    img = cv2.imread(path)
    resized = cv2.resize(img, (224, 224), cv2.INTER_LINEAR) #use cv2.resize(img, (64, 64), cv2.INTER_LINEAR)
    return [path, resized]

def normalize_image_features(paths):
    print('here')
    imf_d = {}
    p = Pool(cpu_count())
    print('here_1')
    ret = p.map(get_im_cv2, paths)
    print('here_2')
    for i in range(len(ret)):
        imf_d[ret[i][0]] = ret[i][1]
    ret = []
    fdata = [imf_d[f] for f in paths]
    fdata = np.array(fdata, dtype=np.uint8)
#     fdata = fdata.transpose((0, 3, 1, 2))
    fdata = fdata.astype('float32')
    fdata = fdata / 255
    return fdata

def subtract_mean(data, mean):
    for sample in data:
        sample[0] -= mean[0]
        sample[1] -= mean[1]
        sample[2] -= mean[2]
    return data


train = glob.glob('train/**/*.jpg') #+ glob.glob('../input/additional/**/*.jpg')
train = pd.DataFrame([[p.split('/')[1],p.split('/')[2],p] for p in train], columns = ['type','image','path'])#[::5] #limit for Kaggle Demo
train = im_stats(train)
train = train[train['size'] != '0 0'].reset_index(drop=True) #corrupt images removed
print("Bad images removed")
print("loading test data")
train_data = normalize_image_features(train['path'])

# print('sustracting mean')

# mean = train_data.mean(axis=(0, 2, 3))
# train_data = subtract_mean(train_data, mean)

print("test data loaded")
np.save('train_vgg.npy', train_data, allow_pickle=True, fix_imports=True)

print("Preparing target")
le = LabelEncoder()
train_target = le.fit_transform(train['type'].values)
print(le.classes_) 
np.save('train_target_vgg.npy', train_target, allow_pickle=True, fix_imports=True)



/home/mwrln/.local/lib/python3.5/site-packages/PIL/TiffImagePlugin.py:692: UserWarning: Possibly corrupt EXIF data.  Expecting to read 524288 bytes but only got 0. Skipping tag 3
  "Skipping tag %s" % (size, len(data), tag))
/home/mwrln/.local/lib/python3.5/site-packages/PIL/TiffImagePlugin.py:692: UserWarning: Possibly corrupt EXIF data.  Expecting to read 524288 bytes but only got 0. Skipping tag 3
  "Skipping tag %s" % (size, len(data), tag))
/home/mwrln/.local/lib/python3.5/site-packages/PIL/TiffImagePlugin.py:692: UserWarning: Possibly corrupt EXIF data.  Expecting to read 524288 bytes but only got 0. Skipping tag 3
  "Skipping tag %s" % (size, len(data), tag))
/home/mwrln/.local/lib/python3.5/site-packages/PIL/TiffImagePlugin.py:692: UserWarning: Possibly corrupt EXIF data.  Expecting to read 524288 bytes but only got 0. Skipping tag 3
  "Skipping tag %s" % (size, len(data), tag))
/home/mwrln/.local/lib/python3.5/site-packages/PIL/TiffImagePlugin.py:692: UserWarning: Possibly cor

/home/mwrln/.local/lib/python3.5/site-packages/PIL/TiffImagePlugin.py:692: UserWarning: Possibly corrupt EXIF data.  Expecting to read 641466368 bytes but only got 0. Skipping tag 4
  "Skipping tag %s" % (size, len(data), tag))
/home/mwrln/.local/lib/python3.5/site-packages/PIL/TiffImagePlugin.py:692: UserWarning: Possibly corrupt EXIF data.  Expecting to read 205389824 bytes but only got 0. Skipping tag 5
  "Skipping tag %s" % (size, len(data), tag))
/home/mwrln/.local/lib/python3.5/site-packages/PIL/TiffImagePlugin.py:692: UserWarning: Possibly corrupt EXIF data.  Expecting to read 497876992 bytes but only got 0. Skipping tag 4
  "Skipping tag %s" % (size, len(data), tag))
/home/mwrln/.local/lib/python3.5/site-packages/PIL/TiffImagePlugin.py:692: UserWarning: Possibly corrupt EXIF data.  Expecting to read 205389824 bytes but only got 0. Skipping tag 5
  "Skipping tag %s" % (size, len(data), tag))
/home/mwrln/.local/lib/python3.5/site-packages/PIL/TiffImagePlugin.py:692: UserWarning: 

/home/mwrln/.local/lib/python3.5/site-packages/PIL/TiffImagePlugin.py:692: UserWarning: Possibly corrupt EXIF data.  Expecting to read 205914112 bytes but only got 12596. Skipping tag 4
  "Skipping tag %s" % (size, len(data), tag))
/home/mwrln/.local/lib/python3.5/site-packages/PIL/TiffImagePlugin.py:692: UserWarning: Possibly corrupt EXIF data.  Expecting to read 205914112 bytes but only got 9897. Skipping tag 4
  "Skipping tag %s" % (size, len(data), tag))
/home/mwrln/.local/lib/python3.5/site-packages/PIL/TiffImagePlugin.py:692: UserWarning: Possibly corrupt EXIF data.  Expecting to read 218497024 bytes but only got 0. Skipping tag 5
  "Skipping tag %s" % (size, len(data), tag))
/home/mwrln/.local/lib/python3.5/site-packages/PIL/TiffImagePlugin.py:692: UserWarning: Possibly corrupt EXIF data.  Expecting to read 218497024 bytes but only got 0. Skipping tag 5
  "Skipping tag %s" % (size, len(data), tag))
/home/mwrln/.local/lib/python3.5/site-packages/PIL/TiffImagePlugin.py:692: UserWa

/home/mwrln/.local/lib/python3.5/site-packages/PIL/TiffImagePlugin.py:692: UserWarning: Possibly corrupt EXIF data.  Expecting to read 464977920 bytes but only got 0. Skipping tag 4
  "Skipping tag %s" % (size, len(data), tag))
/home/mwrln/.local/lib/python3.5/site-packages/PIL/TiffImagePlugin.py:692: UserWarning: Possibly corrupt EXIF data.  Expecting to read 219021312 bytes but only got 12076. Skipping tag 4
  "Skipping tag %s" % (size, len(data), tag))
/home/mwrln/.local/lib/python3.5/site-packages/PIL/TiffImagePlugin.py:692: UserWarning: Possibly corrupt EXIF data.  Expecting to read 205914112 bytes but only got 10237. Skipping tag 4
  "Skipping tag %s" % (size, len(data), tag))
/home/mwrln/.local/lib/python3.5/site-packages/PIL/TiffImagePlugin.py:692: UserWarning: Possibly corrupt EXIF data.  Expecting to read 537722880 bytes but only got 0. Skipping tag 4
  "Skipping tag %s" % (size, len(data), tag))
/home/mwrln/.local/lib/python3.5/site-packages/PIL/TiffImagePlugin.py:692: UserW

/home/mwrln/.local/lib/python3.5/site-packages/PIL/TiffImagePlugin.py:692: UserWarning: Possibly corrupt EXIF data.  Expecting to read 205914112 bytes but only got 11107. Skipping tag 4
  "Skipping tag %s" % (size, len(data), tag))
/home/mwrln/.local/lib/python3.5/site-packages/PIL/TiffImagePlugin.py:692: UserWarning: Possibly corrupt EXIF data.  Expecting to read 765788160 bytes but only got 0. Skipping tag 4
  "Skipping tag %s" % (size, len(data), tag))
/home/mwrln/.local/lib/python3.5/site-packages/PIL/TiffImagePlugin.py:692: UserWarning: Possibly corrupt EXIF data.  Expecting to read 205914112 bytes but only got 10602. Skipping tag 4
  "Skipping tag %s" % (size, len(data), tag))
/home/mwrln/.local/lib/python3.5/site-packages/PIL/TiffImagePlugin.py:692: UserWarning: Possibly corrupt EXIF data.  Expecting to read 587464704 bytes but only got 0. Skipping tag 4
  "Skipping tag %s" % (size, len(data), tag))
/home/mwrln/.local/lib/python3.5/site-packages/PIL/TiffImagePlugin.py:692: UserW

...

/home/mwrln/.local/lib/python3.5/site-packages/PIL/TiffImagePlugin.py:692: UserWarning: Possibly corrupt EXIF data.  Expecting to read 205914112 bytes but only got 11583. Skipping tag 4
  "Skipping tag %s" % (size, len(data), tag))
/home/mwrln/.local/lib/python3.5/site-packages/PIL/TiffImagePlugin.py:692: UserWarning: Possibly corrupt EXIF data.  Expecting to read 479068160 bytes but only got 0. Skipping tag 4
  "Skipping tag %s" % (size, len(data), tag))
/home/mwrln/.local/lib/python3.5/site-packages/PIL/TiffImagePlugin.py:692: UserWarning: Possibly corrupt EXIF data.  Expecting to read 650903552 bytes but only got 0. Skipping tag 4
  "Skipping tag %s" % (size, len(data), tag))
/home/mwrln/.local/lib/python3.5/site-packages/PIL/TiffImagePlugin.py:692: UserWarning: Possibly corrupt EXIF data.  Expecting to read 205914112 bytes but only got 10108. Skipping tag 4
  "Skipping tag %s" % (size, len(data), tag))
/home/mwrln/.local/lib/python3.5/site-packages/PIL/TiffImagePlugin.py:692: UserW

/home/mwrln/.local/lib/python3.5/site-packages/PIL/TiffImagePlugin.py:692: UserWarning: Possibly corrupt EXIF data.  Expecting to read 701300736 bytes but only got 0. Skipping tag 4
  "Skipping tag %s" % (size, len(data), tag))
/home/mwrln/.local/lib/python3.5/site-packages/PIL/TiffImagePlugin.py:692: UserWarning: Possibly corrupt EXIF data.  Expecting to read 205914112 bytes but only got 10558. Skipping tag 4
  "Skipping tag %s" % (size, len(data), tag))
/home/mwrln/.local/lib/python3.5/site-packages/PIL/TiffImagePlugin.py:692: UserWarning: Possibly corrupt EXIF data.  Expecting to read 205914112 bytes but only got 14970. Skipping tag 4
  "Skipping tag %s" % (size, len(data), tag))
/home/mwrln/.local/lib/python3.5/site-packages/PIL/TiffImagePlugin.py:692: UserWarning: Possibly corrupt EXIF data.  Expecting to read 219021312 bytes but only got 12645. Skipping tag 4
  "Skipping tag %s" % (size, len(data), tag))
/home/mwrln/.local/lib/python3.5/site-packages/PIL/TiffImagePlugin.py:692: U

/home/mwrln/.local/lib/python3.5/site-packages/PIL/TiffImagePlugin.py:692: UserWarning: Possibly corrupt EXIF data.  Expecting to read 219021312 bytes but only got 10216. Skipping tag 4
  "Skipping tag %s" % (size, len(data), tag))
/home/mwrln/.local/lib/python3.5/site-packages/PIL/TiffImagePlugin.py:692: UserWarning: Possibly corrupt EXIF data.  Expecting to read 219021312 bytes but only got 10240. Skipping tag 4
  "Skipping tag %s" % (size, len(data), tag))
/home/mwrln/.local/lib/python3.5/site-packages/PIL/TiffImagePlugin.py:692: UserWarning: Possibly corrupt EXIF data.  Expecting to read 651427840 bytes but only got 0. Skipping tag 4
  "Skipping tag %s" % (size, len(data), tag))
/home/mwrln/.local/lib/python3.5/site-packages/PIL/TiffImagePlugin.py:692: UserWarning: Possibly corrupt EXIF data.  Expecting to read 715194368 bytes but only got 0. Skipping tag 4
  "Skipping tag %s" % (size, len(data), tag))
/home/mwrln/.local/lib/python3.5/site-packages/PIL/TiffImagePlugin.py:692: UserW

Bad images removed
loading test data
test data loaded
Preparing target
['Type_1' 'Type_2' 'Type_3']


Process ForkPoolWorker-4:
Process ForkPoolWorker-9:
Process ForkPoolWorker-12:
Process ForkPoolWorker-3:
Process ForkPoolWorker-7:
Process ForkPoolWorker-2:
Process ForkPoolWorker-1:
Process ForkPoolWorker-6:
Process ForkPoolWorker-5:
Process ForkPoolWorker-8:
Process ForkPoolWorker-11:
Traceback (most recent call last):
Traceback (most recent call last):
Process ForkPoolWorker-10:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    s

In [4]:
test = glob.glob('test/*.jpg')
test = pd.DataFrame([[p.split('/')[1],p] for p in test], columns = ['image','path']) #[::20] #limit for Kaggle Demo
print("loading train data")
test_data = normalize_image_features(test['path'])
np.save('test_vgg.npy', test_data, allow_pickle=True, fix_imports=True)
print("test data loaded")
test_id = test.image.values
np.save('test_id_vgg.npy', test_id, allow_pickle=True, fix_imports=True)

loading train data
test data loaded


Process ForkPoolWorker-32:
Process ForkPoolWorker-33:
Process ForkPoolWorker-34:
Process ForkPoolWorker-36:
Process ForkPoolWorker-35:
Process ForkPoolWorker-31:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/usr/lib/python3.5/multiprocessing/process.py", li

In [20]:
test_2 = glob.glob('/mnt/test_stg2/*.jpg')
test_2 = pd.DataFrame([[p.split('/')[1],p] for p in test_2], columns = ['image','path'])
test_2.head()

,image,path
0,mnt,/mnt/test_stg2/10029.jpg
1,mnt,/mnt/test_stg2/12543.jpg
2,mnt,/mnt/test_stg2/13295.jpg
3,mnt,/mnt/test_stg2/12812.jpg
4,mnt,/mnt/test_stg2/11356.jpg


In [2]:
img = cv2.imread('/mnt/test_stg2/10029.jpg')
resized = cv2.resize(img, (224, 224), cv2.INTER_LINEAR)

In [5]:
test_2 = glob.glob('/mnt/test_stg2/*.jpg')
test_2 = pd.DataFrame([[p.split('/')[1],p] for p in test_2], columns = ['image','path']) #[::20] #limit for Kaggle Demo
print("loading train data")
test_2_data = normalize_image_features(test_2['path'])
np.save('/mnt/test_2_vgg.npy', test_2_data, allow_pickle=True, fix_imports=True)
print("test data loaded")
test_2_id = test_2.image.values
np.save('/mnt/test_2_id_vgg.npy', test_2_id, allow_pickle=True, fix_imports=True)

loading train data
here
Processed yet another image
Processed yet another image
here_1
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet ano

Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet anothe

Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet anothe

Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet anothe

Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet anothe

...

Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet anothe

Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet anothe

Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet anothe

Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet anothe

PermissionError: [Errno 13] Permission denied: '/mnt/test_2_vgg.npy'

In [9]:
np.save('test_2_vgg.npy', test_2_data, allow_pickle=True, fix_imports=True)
test_2_id = test_2.image.values
np.save('test_2_id_vgg.npy', test_2_id, allow_pickle=True, fix_imports=True)

In [1]:
from keras.wrappers.scikit_learn import KerasClassifier
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Flatten, Activation
from keras.layers.convolutional import Convolution2D, ZeroPadding2D, MaxPooling2D
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from keras import backend as K
from keras.callbacks import ModelCheckpoint

from keras import applications

# K.set_image_dim_ordering('tf')
K.set_image_data_format('channels_last')
K.set_floatx('float32')

import pandas as pd
import numpy as np

Using TensorFlow backend.


In [7]:
# train_data.shape

In [3]:
# from keras import applications
# from keras import backend as K

# K.set_image_data_format('channels_last')
# vgg16 = applications.VGG16(weights='imagenet', include_top=False, input_shape=(64, 64, 3))

In [3]:
from keras.models import Model
from keras import optimizers
np.random.seed(17)

train_data = np.load('train_vgg.npy')
train_target = np.load('train_target_vgg.npy')
top_model_weights_path = 'vgg_fc_model.h5'
print('Data loaded')
x_train,x_val_train,y_train,y_val_train = train_test_split(train_data,train_target,test_size=0.4, random_state=17)

BATCH_SIZE = 4

vgg16 = applications.VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# vgg16.add(Flatten())
# vgg16.add(Dense(4096, activation='relu'))
# vgg16.add(Dropout(0.5))
# vgg16.add(Dense(4096, activation='relu'))
# vgg16.add(Dropout(0.5))
# vgg16.add(Dense(3, activation='softmax'))


top_model = Sequential()
top_model.add(Flatten(input_shape=vgg16.output_shape[1:]))
top_model.add(Dense(4096, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(4096, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(3, activation='softmax'))

model = Model(input= vgg16.input, output= top_model(vgg16.output))

opt = optimizers.SGD(lr=0.0001, momentum=0.9, decay=1e-4, nesterov=False) #optimizers.Nadam(lr=0.0002)
model.compile(optimizer=opt, loss='sparse_categorical_crossentropy', metrics=['accuracy']) 


print('Loaded vgg16')
# datagen = ImageDataGenerator(rescale=1. / 255, data_format='channels_last')
datagen = ImageDataGenerator(
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        vertical_flip=True,
        fill_mode='nearest')


# generator = datagen.flow(x_train,y_train, batch_size=BATCH_SIZE, shuffle=True)

# bottleneck_features_train = vgg16.predict_generator(
#     generator, len(x_train) // BATCH_SIZE)
# np.save('bottleneck_features_train.npy',
#         bottleneck_features_train, allow_pickle=True, fix_imports=True)


# generator = datagen.flow(x_val_train, y_val_train, batch_size=BATCH_SIZE, shuffle=True)

# bottleneck_features_validation = vgg16.predict_generator(
#     generator, len(x_val_train) // BATCH_SIZE)
# np.save('bottleneck_features_validation.npy',
#         bottleneck_features_validation,allow_pickle=True, fix_imports=True)





Data loaded
Loaded vgg16


/home/mwrln/.local/lib/python3.5/site-packages/ipykernel_launcher.py:31: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("se...)`


In [10]:
BATCH_SIZE = 4

x_train,x_val_train,y_train,y_val_train = train_test_split(train_data,train_target,test_size=0.4, random_state=17)
print("fitting data")
model.fit_generator(datagen.flow(x_train,y_train, batch_size=BATCH_SIZE, shuffle=True), nb_epoch=30, callbacks=[ModelCheckpoint('vgg_model.hdf5', save_best_only=True)], samples_per_epoch=len(x_train), verbose=1, validation_data=(x_val_train, y_val_train))
print("data fitted in model")
test_data = np.load('test_vgg.npy')
test_id = np.load('test_id_vgg.npy')
print("creating predictions")
predictions = model.predict(test_data)
print("predictions made")

df = pd.DataFrame(predictions, columns=['Type_1','Type_2','Type_3'])
df['image_name'] = test_id
df.to_csv('submission_vgg.csv', index=False)

fitting data
Epoch 1/30


/home/mwrln/.local/lib/python3.5/site-packages/ipykernel_launcher.py:5: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras.pre..., validation_data=(array([[[..., steps_per_epoch=222, verbose=1, callbacks=[<keras.ca..., epochs=30)`
  """


222/222 [==============================] - 85s - loss: 0.9469 - acc: 0.5450 - val_loss: 0.9506 - val_acc: 0.5245
Epoch 2/30
222/222 [==============================] - 86s - loss: 0.9368 - acc: 0.5608 - val_loss: 0.9281 - val_acc: 0.5649
Epoch 3/30
222/222 [==============================] - 58s - loss: 0.8929 - acc: 0.5766 - val_loss: 0.9918 - val_acc: 0.5076
Epoch 4/30
222/222 [==============================] - 100s - loss: 0.8935 - acc: 0.5845 - val_loss: 0.9155 - val_acc: 0.5700
Epoch 5/30
222/222 [==============================] - 58s - loss: 0.8603 - acc: 0.6126 - val_loss: 0.9165 - val_acc: 0.5835
Epoch 6/30
222/222 [==============================] - 95s - loss: 0.8669 - acc: 0.6036 - val_loss: 0.8902 - val_acc: 0.5970
Epoch 7/30
222/222 [==============================] - 58s - loss: 0.8617 - acc: 0.5788 - val_loss: 0.9232 - val_acc: 0.5295
Epoch 8/30
222/222 [==============================] - 101s - loss: 0.8417 - acc: 0.6194 - val_loss: 0.8848 - val_acc: 0.5902
Epoch 9/30
222/22

AttributeError: 'Model' object has no attribute 'predict_proba'

In [11]:
test_data = np.load('test_vgg.npy')
test_id = np.load('test_id_vgg.npy')
print("creating predictions")
predictions = model.predict(test_data)
print("predictions made")

df = pd.DataFrame(predictions, columns=['Type_1','Type_2','Type_3'])
df['image_name'] = test_id
df.to_csv('submission_vgg.csv', index=False)

creating predictions
predictions made


In [ ]:
# test_2_data = np.load('test_2_vgg.npy')
# test_2_id = np.load('test_2_id_vgg.npy')
print("creating predictions")
predictions = model.predict(test_2_data)
print("predictions made")

df = pd.DataFrame(predictions, columns=['Type_1','Type_2','Type_3'])
df['image_name'] = test_2_id
df.to_csv('submission_vgg_2.csv', index=False)

creating predictions


In [11]:
from keras.models import load_model

model = load_model('vgg_model.hdf5')

Using TensorFlow backend.


In [7]:
model.layers

In [12]:
def cleanImages():
#     datagen = ImageDataGenerator(rotation_range=0.3, zoom_range=0.3)
    datagen = ImageDataGenerator(
            rotation_range=20,
            width_shift_range=0.2,
            height_shift_range=0.2,
            zoom_range=0.2,
            horizontal_flip=True,
            vertical_flip=True,
            fill_mode='nearest')
    datagen.fit(train_data)
    return datagen

def fitAndPredict():
    print("cleaning images")
    datagen=cleanImages()
    print("images cleaned")
    
    model = create_model()
    x_train,x_val_train,y_train,y_val_train = train_test_split(train_data,train_target,test_size=0.4, random_state=17)
    print("fitting data")
    model.fit_generator(datagen.flow(x_train,y_train, batch_size=32, shuffle=True), nb_epoch=400, callbacks=[ModelCheckpoint('test_save.hdf5', save_best_only=True)], samples_per_epoch=len(x_train), verbose=1, validation_data=(x_val_train, y_val_train))
    print("data fitted in model")
    test_data = np.load('test_add.npy')
    test_id = np.load('test_id_add.npy')
    print("creating predictions")
    predictions = model.predict_proba(test_data)
    print("predictions made")
    return predictions

def createSub():
    pred=fitAndPredict()
    print("creating submission file")
    df = pd.DataFrame(pred, columns=['Type_1','Type_2','Type_3'])
    df['image_name'] = test_id
    df.to_csv('submission_vgg.csv', index=False)
    print("submission created")

In [13]:
if __name__ == '__main__':
    
    createSub()

cleaning images
images cleaned


NameError: name 'create_model' is not defined

In [ ]:
generator = datagen.flow(x_val_train, y_val_train, batch_size=BATCH_SIZE, shuffle=True)

bottleneck_features_validation = vgg16.predict_generator(
    generator, len(x_val_train) // BATCH_SIZE)
np.save('bottleneck_features_validation.npy',
        bottleneck_features_validation,allow_pickle=True, fix_imports=True)

In [8]:
# np.save('bottleneck_features_train.npy',
#         bottleneck_features_train, allow_pickle=True, fix_imports=True)


# generator = datagen.flow(x_val_train, y_val_train, batch_size=BATCH_SIZE, shuffle=True)
# bottleneck_features_validation = vgg16.predict_generator(
#     generator, len(x_val_train) // BATCH_SIZE)
# np.save('bottleneck_features_validation.npy',
#         bottleneck_features_validation,allow_pickle=True, fix_imports=True)

In [9]:
3*3

9

In [21]:
np.random.seed(17)

train_data = np.load('train_vgg.npy')
train_target = np.load('train_target_vgg.npy')
top_model_weights_path = 'bottleneck_fc_model.h5'
print('Data loaded')
x_train,x_val_train,y_train,y_val_train = train_test_split(train_data,train_target,test_size=0.4, random_state=17)

BATCH_SIZE = 4


Data loaded


In [34]:
train_data = np.load('bottleneck_features_train.npy')
train_labels = y_train

validation_data = np.load('bottleneck_features_validation.npy')
validation_labels = y_val_train

top_model_weights_path = 'bottleneck_fc_model.h5'
BATCH_SIZE = 4

model = Sequential()
# model.add(Flatten(input_shape=train_data.shape[1:]))
# model.add(Dropout(0.4))
# model.add(Dense(256, activation='relu'))
# model.add(Dropout(0.7))
# model.add(Dense(3, activation='softmax'))
model.add(Flatten(input_shape=train_data.shape[1:]))
# model.add(Dense(128, activation='tanh'))
model.add(Dropout(0.9))
# model.add(Dense(12, activation='tanh'))
# model.add(Dropout(0.1))
model.add(Dense(3, activation='softmax'))

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.fit(train_data, train_labels,
          epochs=20,
          batch_size=BATCH_SIZE,
#           callbacks=[ModelCheckpoint('vgg_beta.hdf5', save_best_only=True)],
          validation_data=(validation_data, validation_labels))
model.save_weights(top_model_weights_path)


Train on 888 samples, validate on 593 samples
Epoch 1/20
888/888 [==============================] - 0s - loss: 1.8555 - acc: 0.4043 - val_loss: 1.0042 - val_acc: 0.5110
Epoch 2/20
888/888 [==============================] - 0s - loss: 1.4746 - acc: 0.4223 - val_loss: 1.0213 - val_acc: 0.4570
Epoch 3/20
888/888 [==============================] - 0s - loss: 1.4641 - acc: 0.4077 - val_loss: 1.0136 - val_acc: 0.5548
Epoch 4/20
888/888 [==============================] - 0s - loss: 1.2918 - acc: 0.4595 - val_loss: 1.0102 - val_acc: 0.5295
Epoch 5/20
888/888 [==============================] - 0s - loss: 1.3396 - acc: 0.4268 - val_loss: 1.0164 - val_acc: 0.5110
Epoch 6/20
888/888 [==============================] - 0s - loss: 1.3159 - acc: 0.4403 - val_loss: 1.0084 - val_acc: 0.5396
Epoch 7/20
888/888 [==============================] - 0s - loss: 1.2834 - acc: 0.4606 - val_loss: 1.0357 - val_acc: 0.4553
Epoch 8/20
888/888 [==============================] - 0s - loss: 1.2841 - acc: 0.4572 - val_l

In [23]:
test_data = np.load('test_vgg.npy')
test_bottleneck = vgg16.predict(test_data)
test_bottleneck.shape

NameError: name 'test_bottlenect' is not defined

In [26]:
predictions = model.predict_proba(test_bottleneck)
test_id = np.load('test_id_vgg.npy')
df = pd.DataFrame(predictions, columns=['Type_1','Type_2','Type_3'])
df['image_name'] = test_id
df.to_csv('submission.csv', index=False)
print("submission created")

 32/512 [>.............................] - ETA: 2ssubmission created


In [9]:
train_data.shape

(888, 2, 2, 512)

In [20]:
test_data = np.load('test_vgg.npy')
test_data.shape

(512, 64, 64, 3)

In [10]:
validation_data.shape

(593, 2, 2, 512)